<a href="https://colab.research.google.com/github/ssgalitsky/fast-style-transfer/blob/master/colab_fst_blood_hands1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup environment

#### Mount Google Drive
Use Drive as persistant storage for input/output files, test images, checkpoints.

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


#### Compile Caffe

In [0]:
print "Install deps and gcc"

with open('/etc/apt/sources.list') as f:
  txt = f.read()
with open('/etc/apt/sources.list', 'w') as f:
  f.write(txt.replace('# deb-src','deb-src'))
  
t=!apt update
t=!apt build-dep caffe-cuda
t=!apt install g++-5

t=!update-alternatives --remove-all gcc 
t=!update-alternatives --remove-all g++
t=!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 20
t=!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 20
t=!update-alternatives --install /usr/bin/cc cc /usr/bin/gcc 30
t=!update-alternatives --set cc /usr/bin/gcc
t=!update-alternatives --install /usr/bin/c++ c++ /usr/bin/g++ 30
t=!update-alternatives --set c++ /usr/bin/g++

print "Get boost"

t=!wget https://dl.bintray.com/boostorg/release/1.67.0/source/boost_1_67_0.tar.bz2
t=!tar --bzip2 -xf boost_1_67_0.tar.bz2

print "Compile boost"

t=!cd boost_1_67_0 && ./bootstrap.sh --exec-prefix=/usr/local --with-libraries=system,filesystem,regex,thread,python \
--with-python-version=2.7 --with-python-root=/usr
t=!cd boost_1_67_0 && ./b2 install

print "Clone SSD-Caffe and configure it"

t=!git clone https://github.com/weiliu89/caffe.git && cd caffe && git checkout ssd
  
with open('caffe/Makefile.config.example') as f:
  config = f.read()
comment = ['CUDA_DIR := /usr/local/cuda', 
           'BLAS := open']
uncomment = ['# CUDA_DIR := /usr', 
             '# BLAS := atlas', 
             '# OPENCV_VERSION := 3', '# WITH_PYTHON_LAYER := 1'] #
replace = [('INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include',
            'INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include /usr/include/hdf5/serial /usr/local/lib/python2.7/dist-packages/numpy/core/include/'), 
           ('LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib',
            'LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib /usr/lib/x86_64-linux-gnu/hdf5/serial')]
for c in uncomment:
  config = config.replace(c, c[2:])
for c in comment:
  config = config.replace(c, '# '+c)
for c1,c2 in replace:
  config = config.replace(c1, c2)
with open('caffe/Makefile.config', 'w') as f:
  f.write(config)
  
with open('caffe/Makefile') as f:
  mfile = f.read()
with open('caffe/Makefile', 'w') as f:
  f.write(mfile.replace('-isystem','-I'))
  
with open('/usr/include/x86_64-linux-gnu/c++/5/bits/c++config.h') as f:
  txt = f.read()
with open('/usr/include/x86_64-linux-gnu/c++/5/bits/c++config.h', 'w') as f:
  f.write(txt.replace('/* #undef _GLIBCXX_USE_C99_MATH */',
                      '/* #undef _GLIBCXX_USE_C99_MATH */\n#define  _GLIBCXX_USE_C99_MATH  1'))
  
print "Make Caffe"

#t=!cd caffe && make -j8 && make pycaffe && make test -j8 && make distribute
t=!cd caffe && make -j8 && make pycaffe && make distribute
print '\n'.join(t[-10:])

t=!echo /usr/local/lib >> /etc/ld.so.conf && ldconfig
t=!echo /content/caffe/distribute/lib >> /etc/ld.so.conf && ldconfig

print "DONE"

#### Get code & models & training dataset (COCO2017)
[Original code](https://github.com/lengstrom/fast-style-transfer) code by  Logan Engstrom uses COCO2014 Training Dataset.
We'll try more lightweight image collection - COCO2017 Validation Dataset (1GB zip vs 14GB zip).

In [0]:
!git clone https://github.com/ssgalitsky/fast-style-transfer.git

In [8]:

!pip install --upgrade youtube-dl

     |████████████████████████████████| 1.8MB 3.4MB/s 


In [0]:
%cd /content/fast-style-transfer/

In [0]:
!bash setup.sh


#### Misc helper procedures

In [0]:
        
#import cv2
import matplotlib.pyplot as plt
import numpy as np

def show_image_grid(images, n_images=10, n_rows=3, figsize=(10, 10), randomize=False):
    n_cols = int(np.ceil(n_images / n_rows))

    fig = plt.figure(figsize=figsize)

    for i in range(n_images):
        rnd = i
        if randomize:
            rnd = np.random.randint(0, len(images))
        image = images[rnd]
        ax = fig.add_subplot(n_rows, n_cols, i + 1)
        ax.imshow(image)
        ax.set_yticks([])
        ax.set_xticks([])
        

In [0]:
#import os
#import subprocess
import youtube_dl

def youtubeDownload(url, filename,formats="18"):
    youtube_downloader_params = {"quiet": False, "outtmpl": filename, "format": formats}
    with youtube_dl.YoutubeDL(params=youtube_downloader_params) as ydl:
        ydl.download([url])
        

        

### Processing

#### Train model

##### Training mode arguments

style.py
style.py trains networks that can transfer styles from artwork into images.

Flags

--checkpoint-dir: Directory to save checkpoint in. Required.

--style: Path to style image. Required.

--train-path: Path to training images folder. Default: data/train2014.

--test: Path to content image to test network on at at every checkpoint iteration. Default: no image.

--test-dir: Path to directory to save test images in. Required if --test is passed a value.

--epochs: Epochs to train for. Default: 2.

--batch_size: Batch size for training. Default: 4.

--checkpoint-iterations: Number of iterations to go for between checkpoints. Default: 2000.

--vgg-path: Path to VGG19 network (default). Can pass VGG16 if you want to try out other loss functions. Default: data/imagenet-vgg-verydeep-19.mat.

--content-weight: Weight of content in loss function. Default: 7.5e0.

--style-weight: Weight of style in loss function. Default: 1e2.

--tv-weight: Weight of total variation term in loss function. Default: 2e2.

--learning-rate: Learning rate for optimizer. Default: 1e-3.

--slow: For debugging loss function. Direct optimization on pixels using Gatys' approach. Uses test image as content value, test_dir for saving fully optimized images.

##### Run training

In [0]:
styleUrl="https://i.ebayimg.com/images/i/123369606193-0-1/s-l1000.jpg"
testUrl="https://24smi.org/public/media/2017/10/31/07.jpg"
suffice="FAV/blood_hands_1"

In [0]:
!mkdir -p /gdrive/My\ Drive/{suffice}/checkpoint /gdrive/My\ Drive/{suffice}/test

In [33]:

!wget {styleUrl} -O /gdrive/My\ Drive/{suffice}/style.jpg
!wget {testUrl} -O /gdrive/My\ Drive/{suffice}/test.jpg






--2019-05-09 07:24:20--  https://i.ebayimg.com/images/i/123369606193-0-1/s-l1000.jpg
Resolving i.ebayimg.com (i.ebayimg.com)... 23.202.234.54, 2600:1407:16:3bc::24d4, 2600:1407:16:3b6::24d4
Connecting to i.ebayimg.com (i.ebayimg.com)|23.202.234.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122250 (119K) [image/jpeg]
Saving to: ‘/gdrive/My Drive/FAV/blood_hands_1/style.jpg’

/gdrive/My Drive/FA 100%[===================>] 119.38K  --.-KB/s    in 0.02s   

2019-05-09 07:24:20 (5.33 MB/s) - ‘/gdrive/My Drive/FAV/blood_hands_1/style.jpg’ saved [122250/122250]

--2019-05-09 07:24:22--  https://24smi.org/public/media/2017/10/31/07.jpg
Resolving 24smi.org (24smi.org)... 37.48.106.245
Connecting to 24smi.org (24smi.org)|37.48.106.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223136 (218K) [image/jpeg]
Saving to: ‘/gdrive/My Drive/FAV/blood_hands_1/test.jpg’

/gdrive/My Drive/FA 100%[===================>] 217.91K   528KB/s    in 0.4s

In [0]:
!python style.py --style /gdrive/My\ Drive/{suffice}/style.jpg \
  --checkpoint-dir /gdrive/My\ Drive/{suffice}/checkpoint \
  --test /gdrive/My\ Drive/{suffice}/test.jpg \
  --test-dir /gdrive/My\ Drive/{suffice}/test  \
  --content-weight 1.5e1 \
  --checkpoint-iterations 200 \
  --batch-size 14

Train set has been trimmed slightly..
(1, 1000, 844, 3)
2019-05-09 07:25:10.480818: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-09 07:25:10.481194: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5560df005760 executing computations on platform Host. Devices:
2019-05-09 07:25:10.481233: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-09 07:25:10.710214: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-09 07:25:10.710883: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5560df005340 executing computations on platform CUDA. Devices:
2019-05-09 07:25:10.710924: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-09 07:25:10.711366:

#### Stylize video

##### Download youtube video 

In [0]:
lenin1="https://www.youtube.com/watch?v=qB39bMYnuU4"
youtubeDownload(lenin1, "/gdrive/My Drive/%s/video_in.mp4" % (suffice,),formats="18")

##### Stylize parameters info


--checkpoint-dir: Directory or ckpt file to load checkpoint from. Required.

--in-path: Path to video to transfer style to. Required.

--out-path: Path to out video. Required.

--tmp-dir: Directory to put temporary processing files in. Will generate a dir if you do not pass it a path. Will delete tmpdir afterwards. Default: randomly generates invisible dir, then deletes it after execution completion.

--device: Device to evaluate frames with. Default: /gpu:0.

--batch-size: Batch size for evaluating images. Default: 4.

##### Run stylize on downloaded video file

In [20]:
#gdrive/My Drive/FAV/checkpoint_1/checkpoint

!python transform_video.py --in-path /gdrive/My\ Drive/{suffice}/video_in.mp4 \
  --checkpoint /gdrive/My\ Drive/{suffice}/checkpoint \
  --out-path /gdrive/My\ Drive/{suffice}/video_out.mp4 \
  --device /gpu:0 \
  --batch-size 4


2019-05-09 06:47:22.755435: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-05-09 06:47:22.755678: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x559b0ec9d600 executing computations on platform Host. Devices:
2019-05-09 06:47:22.755706: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-05-09 06:47:23.000926: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-09 06:47:23.001537: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x559b0ec9d1e0 executing computations on platform CUDA. Devices:
2019-05-09 06:47:23.001575: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2019-05-09 06:47:23.002048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433